In [3]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re 
import torch
import wandb


from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.preprocessing import LabelEncoder

from transformers import DistilBertTokenizer, DistilBertModel

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

/opt/homebrew/Caskroom/miniforge/base/envs/jobfit_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /teamspace/studios/this_studio/.netrc


True

In [10]:
# def preprocess(sentence):
#     sentence=str(sentence)
#     sentence = sentence.lower()
#     sentence=sentence.replace('{html}',"")
#     cleanr = re.compile('<.*?>')
#     cleantext = re.sub(cleanr, '', sentence)
#     rem_url=re.sub(r'http\S+', '',cleantext)
#     rem_num = re.sub('[0-9]+', '', rem_url)
#     tokenizer = RegexpTokenizer(r'\w+')
#     tokens = tokenizer.tokenize(rem_num)  
#     filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
#     stem_words=[stemmer.stem(w) for w in filtered_words]
#     lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
#     return " ".join(filtered_words)

# print("--------PROCESSING TRAINING DATA--------")
# processed_df = df.copy()
# processed_df['resume_text'] = processed_df['resume_text'].map(lambda s: preprocess(s))
# processed_df['job_description_text'] = processed_df['job_description_text'].map(lambda s: preprocess(s))
# processed_df['combined_text'] = processed_df['resume_text'] + ' ' + processed_df['job_description_text']
# processed_df['label'] = le.fit_transform(processed_df['label'])

# print("--------PROCESSING TEST DATA--------")
# test_df = pd.read_csv('data/test.csv')
# processed_test_df = test_df.copy()

# processed_test_df['resume_text'] = processed_test_df['resume_text'].map(lambda s: preprocess(s))
# processed_test_df['job_description_text'] = processed_test_df['job_description_text'].map(lambda s: preprocess(s))
# processed_test_df['combined_text'] = processed_test_df['resume_text'] + ' ' + processed_test_df['job_description_text']
# processed_test_df['label'] = le.transform(processed_test_df['label'])

In [11]:
# processed_df.to_csv('data/processed_train.csv', index=False)
# processed_test_df.to_csv('data/processed_test.csv', index=False)

In [12]:
# # Making an evaluation set from the training set
# from sklearn.model_selection import train_test_split

# df_processed = pd.read_csv('data/processed_train.csv')
# train_data, eval_data = train_test_split(df_processed, test_size=0.2, random_state=42)
# train_data.to_csv('data/processed_train.csv', index=False)
# eval_data.to_csv('data/processed_eval.csv', index=False)

In [4]:
processed_df = pd.read_csv('../data/processed_train.csv')
processed_eval = pd.read_csv('../data/processed_eval.csv')
processed_test_df = pd.read_csv('../data/processed_test.csv')

In [30]:
from transformers import BertModel

In [ ]:
# # Load a pretrained model (BERT in this case) and its tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [25]:
class SentencePairDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        """
        df: Pandas DataFrame with columns 'resume_text', 'job_description_text', and 'label'
        """
        self.sentences_a = df['resume_text'].tolist()
        self.sentences_b = df['job_description_text'].tolist()
        self.labels = df['label'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __str__(self):
        string = ''
        string += 'Dataset with {} samples\n'.format(len(self.labels))
        string += 'Sample:\n'
        string += 'Sample 1 Resume: {}\n'.format(self.sentences_a[0])
        string += 'Sample 1 Job Description: {}\n'.format(self.sentences_b[0])
        string += 'Length of Sample 1: {}\n'.format(len(self.sentences_b[0].split()))
        string += 'Label: {}\n'.format(self.labels[0])
        return string

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        encoding_a = self.tokenizer(
            self.sentences_a[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        encoding_b = self.tokenizer(
            self.sentences_b[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        item = {
            'input_ids_a': encoding_a['input_ids'].squeeze(0),
            'attention_mask_a': encoding_a['attention_mask'].squeeze(0),
            'input_ids_b': encoding_b['input_ids'].squeeze(0),
            'attention_mask_b': encoding_b['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }
        return item

In [26]:
len(processed_df.iloc[1,0])

1483

In [27]:
print(SentencePairDataset(processed_df, tokenizer))

Dataset with 4992 samples
Sample:
Sample 1 Resume: summarya business management graduate significant experience disability services human resources finance department seeking apply abilities position human resource department proven ability deal multiple tasks efficiently maintain organization highlightsextensive customer service skillsable retain confidentiality paperwork informationflexible team player quick learner interested new technologiesable meet deadlines handle stressful situations professional mannermicrosoft office word excel access outlookknowledge state accounting system mmars system people soft system client tracking systemexcellent communicator written verbaloffice experience ability create reports analyze data manipulate data experienceaccountant topresentfreeport mcmoran copper gold inc tucson provide administrative support finance unit maintain orderly filing system various departmental units oversee maintain proper appropriate systems storing financial records docum

In [28]:
# ----------- DATA LOADERS -----------------

train_dataset = SentencePairDataset(processed_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

eval_dataset = SentencePairDataset(processed_eval, tokenizer)
eval_loader = DataLoader(eval_dataset, batch_size=4, shuffle=False)

test_dataset = SentencePairDataset(processed_test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [29]:
class SBERT(nn.Module):
    def __init__(self, bert_model):
        super(SBERT, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(self.bert.config.hidden_size * 3, 3)  # 3 classes for classification

    def forward(self, input_ids_a, attention_mask_a, input_ids_b, attention_mask_b):
        outputs_a = self.bert(input_ids_a, attention_mask=attention_mask_a)
        pooled_output_a = outputs_a.last_hidden_state[:, 0, :]  # CLS token for sentence A

        outputs_b = self.bert(input_ids_b, attention_mask=attention_mask_b)
        pooled_output_b = outputs_b.last_hidden_state[:, 0, :]  # CLS token for sentence B

        abs_diff = torch.abs(pooled_output_a - pooled_output_b)
        combined = torch.cat((pooled_output_a, pooled_output_b, abs_diff), dim=1)
        logits = self.fc(combined)

        return logits  # Return raw logits; softmax will be applied in loss calculation

In [19]:
# Check for available devices and assign the appropriate one
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [20]:
print(device)

cuda


In [21]:
# Initialize SBERT model and send it to the selected device
sbert = SBERT(model).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(sbert.parameters(), lr=2e-5)

In [4]:
# Define and initialize wandb
config = {
    "batch_size": 4,
    "learning_rate": 2e-5,
    "epochs": 20,
    "model": "distilbert-base-uncased",
    "dataset": "processed_train",
    "optimizer": "Adam",
    "loss_function": "CrossEntropyLoss",
}

run = wandb.init(
    project="jobfit",
    config=config,
    name="sbert-training-1",
    reinit=True
)

# Access the config
config = wandb.config

wandb: Currently logged in as: grandhi1 (makeai). Use `wandb login --relogin` to force relogin


In [22]:
# Training loop with eval set included
num_epochs = 20
for epoch in range(num_epochs):
    sbert.train()
    epoch_loss = 0
    for batch in train_loader:
        input_ids_a = batch['input_ids_a'].to(device)
        attention_mask_a = batch['attention_mask_a'].to(device)
        input_ids_b = batch['input_ids_b'].to(device)
        attention_mask_b = batch['attention_mask_b'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = sbert(input_ids_a, attention_mask_a, input_ids_b, attention_mask_b)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": loss / len(train_loader)
    })

    print(f"Epoch {epoch + 1}, Train Loss: {epoch_loss:.4f}")

    # Evaluation on eval set
    sbert.eval()
    eval_loss = 0
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for batch in eval_loader:
            input_ids_a = batch['input_ids_a'].to(device)
            attention_mask_a = batch['attention_mask_a'].to(device)
            input_ids_b = batch['input_ids_b'].to(device)
            attention_mask_b = batch['attention_mask_b'].to(device)
            labels = batch['labels'].to(device)

            outputs = sbert(input_ids_a, attention_mask_a, input_ids_b, attention_mask_b)
            loss = loss_fn(outputs, labels)
            eval_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)
    
    eval_accuracy = correct_predictions / total_predictions
    wandb.log({
        "epoch": epoch + 1,
        "eval_loss": eval_loss,
        "eval_accuracy": eval_accuracy
    })
    print(f"Epoch {epoch + 1}, Eval Loss: {eval_loss:.4f}, Eval Accuracy: {eval_accuracy:.4f}")

Epoch 1, Train Loss: 1129.0325
Epoch 1, Eval Loss: 223.4205, Eval Accuracy: 0.6541
Epoch 2, Train Loss: 828.9441
Epoch 2, Eval Loss: 190.7120, Eval Accuracy: 0.7318
Epoch 3, Train Loss: 709.2908
Epoch 3, Eval Loss: 169.2036, Eval Accuracy: 0.7518
Epoch 4, Train Loss: 605.6866
Epoch 4, Eval Loss: 160.8488, Eval Accuracy: 0.7870
Epoch 5, Train Loss: 512.0554
Epoch 5, Eval Loss: 152.7054, Eval Accuracy: 0.7974
Epoch 6, Train Loss: 432.3302
Epoch 6, Eval Loss: 139.5454, Eval Accuracy: 0.8247
Epoch 7, Train Loss: 368.8202
Epoch 7, Eval Loss: 127.9034, Eval Accuracy: 0.8519
Epoch 8, Train Loss: 324.3971
Epoch 8, Eval Loss: 130.0263, Eval Accuracy: 0.8495
Epoch 9, Train Loss: 282.1511
Epoch 9, Eval Loss: 124.1237, Eval Accuracy: 0.8575
Epoch 10, Train Loss: 246.2789
Epoch 10, Eval Loss: 126.7270, Eval Accuracy: 0.8591
Epoch 11, Train Loss: 213.1448
Epoch 11, Eval Loss: 119.4215, Eval Accuracy: 0.8719
Epoch 12, Train Loss: 202.7066
Epoch 12, Eval Loss: 132.6694, Eval Accuracy: 0.8711
Epoch 13,

In [24]:
# Final test performance check
sbert.eval()
test_loss = 0
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids_a = batch['input_ids_a'].to(device)
        attention_mask_a = batch['attention_mask_a'].to(device)
        input_ids_b = batch['input_ids_b'].to(device)
        attention_mask_b = batch['attention_mask_b'].to(device)
        labels = batch['labels'].to(device)

        outputs = sbert(input_ids_a, attention_mask_a, input_ids_b, attention_mask_b)
        loss = loss_fn(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

test_accuracy = correct_predictions / total_predictions
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

wandb.log({
    "test_loss": test_loss,
    "test_accuracy": test_accuracy 
})

Test Loss: 752.4419, Test Accuracy: 0.5458


In [25]:
run.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
eval_accuracy,▁▃▄▅▅▆▇▇▇▇██▇███████
eval_loss,█▆▄▄▃▂▂▂▁▁▁▂▂▁▂▃▂▂▃▂
test_accuracy,▁
test_loss,▁
train_loss,█▄█▇▅▆▃▄▂▃▁▃▃▂▁▁▂▁▁▁
epoch,20
eval_accuracy,0.88231
eval_loss,133.93529
test_accuracy,0.54576
test_loss,752.44194


The cell below can be used to checkpoint/save the weights of the model.

In [ ]:
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
# }, "model_and_optimizer_nn_1.pth")


# Load both model and optimizer state_dicts
# checkpoint = torch.load("model_and_optimizer.pth")
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])